In [1]:
#19th March,2024
#20th March,2024:modified the code 

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import pybedtools

path1=('S012.xlsx')
colnames1=('chr','x1','x2')
df1=pd.read_excel('S012.xlsx',names=colnames1)

path2=('HPVint.xlsx')
colnames2=('sample ID', 'virus type', 'virus start','virus stop', 'Chr.','chr. start', 'chr. stop', 'strand1', 'strand2', '#supporting pairs', '#split read support', 'representative break', 'nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)')
ori_df2=pd.read_excel('HPVint.xlsx',names=colnames2)
ori_df2

# First, let's filter df2 to include only those rows where 'representative break' equals 'Y'                                                                                                                                                              
filtered_df = ori_df2[ori_df2['representative break'] == 'Y']
filtered_df




# Assuming filtered_df is the DataFrame after applying your conditions related to 'representative break'
df2 = filtered_df[['Chr.', 'chr. start', 'chr. stop', 'sample ID']]
df2 = df2.reset_index(drop=True)
df2

/tmp/ipykernel_18155/2826895709.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Chr.  chr. start  chr. stop sample ID
0    chr17     1104459    1104460   GS18001
1     chr2     9961582    9961583   GS18001
2    chr21    11122389   11122390   GS18001
3     chr5    29687633   29687634   GS18001
4     chr5    91748270   91748271   GS18001
..     ...         ...        ...       ...
313   chr9     5437959    5437960   GS21051
314   chr1   226954998  226954999   GS21052
315  chr10    18916383   18916384   GS21052
316   chr3     2140200    2140201   GS21052
317   chr8   129783851  129783871   GS21052

[318 rows x 4 columns]

In [4]:
import pybedtools
from pybedtools import BedTool

# Harmonize chromosome naming in df1 to add 'chr' prefix if missing
df1['chr'] = df1['chr'].apply(lambda x: 'chr' + str(x) if not str(x).startswith('chr') else str(x))

# Do the same for df2
df2['Chr.'] = df2['Chr.'].apply(lambda x: 'chr' + str(x) if not str(x).startswith('chr') else str(x))

bed1 = BedTool.from_dataframe(df1[['chr', 'x1', 'x2']])
bed2 = BedTool.from_dataframe(df2[['Chr.', 'chr. start', 'chr. stop']])



#  Sort the BedTool objects 
sorted_bed1 = bed1.sort()
sorted_bed2 = bed2.sort()

overlap = sorted_bed1.intersect(sorted_bed2, wa=True, wb=True)

# To save the overlap results to a file
overlap.saveas('TAD_HPV_integration_overlap_S012.bed')
# convert to DataFrame
overlap_df = overlap.to_dataframe(names=['TAD_chr', 'TAD_start', 'TAD_end', 
                                         'HPV_chr', 'HPV_start', 'HPV_end'])

print(overlap_df)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
0     chr1    2710000    3520000    chr1    2786393    2786394
1     chr1    9330000    9540000    chr1    9435159    9435160
2     chr1   16110000   16310000    chr1   16245129   16245130
3     chr1   16400000   16530000    chr1   16461227   16461228
4     chr1  199640000  200500000    chr1  200457000  200457001
..     ...        ...        ...     ...        ...        ...
77    chr9    5440000    5600000    chr9    5489910    5489911
78    chr9    5440000    5600000    chr9    5542796    5542797
79    chr9  108650000  109940000    chr9  109336186  109336187
80    chr9  110180000  111560000    chr9  110354690  110354691
81    chr9  131880000  132380000    chr9  131937709  131937710

[82 rows x 6 columns]


In [5]:
### Identifying integrations in multiple TADs and merging those TADs in a single TAD

grouped = overlap_df.groupby(['HPV_chr', 'HPV_start', 'HPV_end'])

#### Aggregate to find the minimum and maximum TAD coordinates for each HPV integration


# Aggregate to find the new TAD boundaries
new_tad_boundaries = grouped.agg(TAD_chr=('TAD_chr', 'first'), 
                                 new_TAD_start=('TAD_start', 'min'), 
                                 new_TAD_end=('TAD_end', 'max')).reset_index()

# This renaming aligns with your intended updates without requiring a suffix
new_tad_boundaries.rename(columns={'new_TAD_start': 'TAD_start_new', 'new_TAD_end': 'TAD_end_new'}, inplace=True)
merged_info = pd.merge(overlap_df, new_tad_boundaries, 
                       on=['HPV_chr', 'HPV_start', 'HPV_end'], 
                       how='left', 
                       suffixes=('', '_new'))
# Directly update 'TAD_start' and 'TAD_end' in 'overlap_df' with the new values
merged_info['TAD_start'] = merged_info['TAD_start_new']
merged_info['TAD_end'] = merged_info['TAD_end_new']

# Drop the temporary new columns used for the update
merged_info.drop(['TAD_start_new', 'TAD_end_new'], axis=1, inplace=True)
merged_info

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end TAD_chr_new
0     chr1    2710000    3520000    chr1    2786393    2786394        chr1
1     chr1    9330000    9540000    chr1    9435159    9435160        chr1
2     chr1   16110000   16310000    chr1   16245129   16245130        chr1
3     chr1   16400000   16530000    chr1   16461227   16461228        chr1
4     chr1  199640000  200500000    chr1  200457000  200457001        chr1
..     ...        ...        ...     ...        ...        ...         ...
77    chr9    5440000    5600000    chr9    5489910    5489911        chr9
78    chr9    5440000    5600000    chr9    5542796    5542797        chr9
79    chr9  108650000  109940000    chr9  109336186  109336187        chr9
80    chr9  110180000  111560000    chr9  110354690  110354691        chr9
81    chr9  131880000  132380000    chr9  131937709  131937710        chr9

[82 rows x 7 columns]

In [6]:
####Check if 'TAD_chr_new' is identical to 'TAD_chr' in all rows
columns_are_identical = (merged_info['TAD_chr'] == merged_info['TAD_chr_new']).all()

print(f"Is 'TAD_chr_new' identical to 'TAD_chr' in all rows? {columns_are_identical}")

Is 'TAD_chr_new' identical to 'TAD_chr' in all rows? True


In [7]:
merged_info.drop('TAD_chr_new', axis=1, inplace=True)


In [8]:
merged_info.iloc[[72,73,74,75,76]]

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
72    chr8  127830000  130750000    chr8  129783851  129783871
73    chr8  127830000  130750000    chr8  129783861  129783862
74    chr8  127830000  130750000    chr8  128384641  128384642
75    chr8  127830000  130750000    chr8  128583547  128583548
76    chr9    5440000    5600000    chr9    5464244    5464245

In [9]:
overlap_df.iloc[[72,73,74,75,76]]

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
72    chr8  127830000  130750000    chr8  129783851  129783871
73    chr8  127830000  130750000    chr8  129783861  129783862
74    chr8  127860000  128710000    chr8  128384641  128384642
75    chr8  127860000  128710000    chr8  128583547  128583548
76    chr9    5440000    5600000    chr9    5464244    5464245

In [10]:
# Ensure both DataFrames are aligned (same index and sorted in the same order if necessary)
# For this comparison, the indices of the rows being compared should correspond to the same data points

# Create a boolean mask for rows where 'TAD_start' or 'TAD_end' are different between the two DataFrames
diff_mask = (merged_info['TAD_start'] != overlap_df['TAD_start']) | (merged_info['TAD_end'] != overlap_df['TAD_end'])
 
# Apply the mask to get the rows from merged_info where 'TAD_start' or 'TAD_end' are different
rows_with_diff = merged_info[diff_mask]

print(rows_with_diff)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
74    chr8  127830000  130750000    chr8  128384641  128384642
75    chr8  127830000  130750000    chr8  128583547  128583548


In [11]:
# Using the previously created diff_mask to select rows from overlap_df
original_rows_with_diff = overlap_df[diff_mask]

print(original_rows_with_diff)

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
74    chr8  127860000  128710000    chr8  128384641  128384642
75    chr8  127860000  128710000    chr8  128583547  128583548


In [12]:
merged_info

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end
0     chr1    2710000    3520000    chr1    2786393    2786394
1     chr1    9330000    9540000    chr1    9435159    9435160
2     chr1   16110000   16310000    chr1   16245129   16245130
3     chr1   16400000   16530000    chr1   16461227   16461228
4     chr1  199640000  200500000    chr1  200457000  200457001
..     ...        ...        ...     ...        ...        ...
77    chr9    5440000    5600000    chr9    5489910    5489911
78    chr9    5440000    5600000    chr9    5542796    5542797
79    chr9  108650000  109940000    chr9  109336186  109336187
80    chr9  110180000  111560000    chr9  110354690  110354691
81    chr9  131880000  132380000    chr9  131937709  131937710

[82 rows x 6 columns]

In [13]:
import pandas as pd

# Assuming your DataFrame is named df

# Identify unique TAD regions and assign a unique ID to each
unique_tads = merged_info[['TAD_chr', 'TAD_start', 'TAD_end']].drop_duplicates().reset_index(drop=True)
unique_tads['TAD_id'] = ['TAD' + str(i).zfill(4) for i in range(284, 284 + len(unique_tads))]

# Create a mapping from each unique TAD region to its TAD ID
tad_id_mapping = unique_tads.set_index(['TAD_chr', 'TAD_start', 'TAD_end'])['TAD_id'].to_dict()

# Map the TAD IDs back to the original DataFrame based on the TAD region
merged_info['TAD_id'] = merged_info.apply(lambda row: tad_id_mapping[(row['TAD_chr'], row['TAD_start'], row['TAD_end'])], axis=1)

# Display the DataFrame to verify the TAD_id assignment
print(merged_info)
merged_info.to_excel('merged_info_S012.xlsx')

   TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end   TAD_id
0     chr1    2710000    3520000    chr1    2786393    2786394  TAD0284
1     chr1    9330000    9540000    chr1    9435159    9435160  TAD0285
2     chr1   16110000   16310000    chr1   16245129   16245130  TAD0286
3     chr1   16400000   16530000    chr1   16461227   16461228  TAD0287
4     chr1  199640000  200500000    chr1  200457000  200457001  TAD0288
..     ...        ...        ...     ...        ...        ...      ...
77    chr9    5440000    5600000    chr9    5489910    5489911  TAD0342
78    chr9    5440000    5600000    chr9    5542796    5542797  TAD0342
79    chr9  108650000  109940000    chr9  109336186  109336187  TAD0343
80    chr9  110180000  111560000    chr9  110354690  110354691  TAD0344
81    chr9  131880000  132380000    chr9  131937709  131937710  TAD0345

[82 rows x 7 columns]


In [14]:
# Merging filtered_df with merged_info_df_S009 to associate sample ID and nearby genes with TADs
merged_info_with_samples = pd.merge(merged_info, 
                                    filtered_df[['Chr.', 'chr. start', 'chr. stop', 'sample ID', 'virus type','nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)']],
                                    left_on=['HPV_chr', 'HPV_start', 'HPV_end'], 
                                    right_on=['Chr.', 'chr. start', 'chr. stop'],
                                    how='left')

merged_info_with_samples
merged_info_with_samples=merged_info_with_samples.drop_duplicates()

In [15]:
merged_info_with_samples.drop_duplicates()
merged_info_with_samples

TAD_chr  TAD_start    TAD_end HPV_chr  HPV_start    HPV_end   TAD_id  Chr.  \
0     chr1    2710000    3520000    chr1    2786393    2786394  TAD0284  chr1   
1     chr1    9330000    9540000    chr1    9435159    9435160  TAD0285  chr1   
2     chr1   16110000   16310000    chr1   16245129   16245130  TAD0286  chr1   
3     chr1   16400000   16530000    chr1   16461227   16461228  TAD0287  chr1   
4     chr1  199640000  200500000    chr1  200457000  200457001  TAD0288  chr1   
..     ...        ...        ...     ...        ...        ...      ...   ...   
77    chr9    5440000    5600000    chr9    5489910    5489911  TAD0342  chr9   
78    chr9    5440000    5600000    chr9    5542796    5542797  TAD0342  chr9   
79    chr9  108650000  109940000    chr9  109336186  109336187  TAD0343  chr9   
80    chr9  110180000  111560000    chr9  110354690  110354691  TAD0344  chr9   
81    chr9  131880000  132380000    chr9  131937709  131937710  TAD0345  chr9   

    chr. start  chr. stop sample ID virus type  \
0      2786393    2786394   GS18076      HPV16   
1      9435159    9435160   GS18051      HPV16   
2     16245129   16245130   GS21009      HPV16   
3     16461227   16461228   GS18005      HPV16   
4    200457000  200457001   GS18071      HPV16   
..         ...        ...       ...        ...   
77     5489910    5489911   GS18088      HPV16   
78     5542796    5542797   GS21036      HPV16   
79   109336186  109336187   GS18026      HPV16   
80   110354690  110354691   GS18010      HPV16   
81   131937709  131937710   GS18062      HPV16   

   nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)  
0                       TTC34,ACTRT2,LINC00982,PRDM16                                 
1                    SPSB1,LOC100506022,H6PD,SLC25A33                                 
2                       SPEN, ZBTB17, FLJ37453, SRARP                                 
3                       EPHA2,FAM131C,ARHGEF19,CLCNKB                                 
4                        KIF14,ZNF281,LINC00862,DDX59                                 
..                                                ...                                 
77                         CD274,PDCD1LG2,PLGRKT,RIC1                                 
78                         PDCD1LG2,CD274,RIC1,PLGRKT                                 
79                 MIR8081,LINC01505,ZNF462,LOC340512                                 
80                    KLF4,LINC01509,RAD23B,LOC340512                                 
81                             IER5L,PTPA,CRAT,DOLPP1                                 

[80 rows x 13 columns]

In [16]:
# Count unique samples per TAD
sample_counts_per_TAD = merged_info_with_samples.groupby('TAD_id')['sample ID'].nunique()

# Identify TAD IDs with at least 3 samples
tads_of_interest_ids = sample_counts_per_TAD[sample_counts_per_TAD >= 3].index

# Filter for TADs of interest based on the identified TAD IDs
tads_of_interest = merged_info_with_samples[merged_info_with_samples['TAD_id'].isin(tads_of_interest_ids)]
if not tads_of_interest_ids.empty:
    print("TADs of interest based on having at least 3 samples:")
    print(tads_of_interest_ids)
else:
    print("No TAD of interest is found")

TADs of interest based on having at least 3 samples:
Index(['TAD0301', 'TAD0328', 'TAD0329', 'TAD0332', 'TAD0341', 'TAD0342'], dtype='object', name='TAD_id')


In [17]:
merged_info_with_samples.to_excel('merged_info_with_samples_S012.xlsx')